<a href="https://colab.research.google.com/github/mertege/FMCW-Data-Classification-/blob/main/Range_Doppler_and_Spectrogram_Multi_Class_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Burada fast, slow ve slow with pocket multi-class olarak karşılaştırıldı.
# MixUp augmentation yapıldı.
# 5 defa run edildi ve değerler aşağıda paylaşıldı.
# Mean test accuracy is 0.65, mean test f1 score is 0.12, max test accuracy is 0.68, max test f1 score is 0.14, 
# Min test accuracy is 0.60, min test f1 score is 0.10, std of test accuracy is 0.03, std of test f1 score is 0.02
# Time elapsed through all process: 3897.19, sec

In [ ]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K 
import gc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get Range-Doppler data from
range_doppler_fast_resized = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_fast_resized.mat')
range_doppler_fast_resized = range_doppler_fast_resized['range_doppler_fast_resized']
range_doppler_fast_resized = np.transpose(range_doppler_fast_resized, (2, 0, 1))
# range_doppler_fast_resized = np.delete(range_doppler_fast_resized,(49), axis=0) # 50th row is deleted since there is no 50th row in spectrogram fast data.
range_doppler_fast_label = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_fast_label.mat')
range_doppler_fast_label = range_doppler_fast_label['range_doppler_fast_label']  

range_doppler_slow_resized = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_slow_resized.mat')
range_doppler_slow_resized = range_doppler_slow_resized['range_doppler_slow_resized']
range_doppler_slow_resized = np.transpose(range_doppler_slow_resized, (2, 0, 1))
range_doppler_slow_label = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_slow_label.mat')
range_doppler_slow_label = range_doppler_slow_label['range_doppler_slow_label']  

range_doppler_slow_pocket_resized = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_slow_pocket_resized.mat')
range_doppler_slow_pocket_resized = range_doppler_slow_pocket_resized['range_doppler_slow_pocket_resized']
range_doppler_slow_pocket_resized = np.transpose(range_doppler_slow_pocket_resized, (2, 0, 1))
range_doppler_pocket_label = scipy.io.loadmat('/content/drive/MyDrive/range_doppler_pocket_label.mat')
range_doppler_pocket_label = range_doppler_pocket_label['range_doppler_pocket_label']  
# Get Range-Doppler data from
spectrogram_fast_resized = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_fast_resized.mat')
spectrogram_fast_resized = spectrogram_fast_resized['spectrogram_fast_resized']
spectrogram_fast_resized = np.transpose(spectrogram_fast_resized, (2, 0, 1))
spectrogram_fast_label = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_fast_label.mat')
spectrogram_fast_label = spectrogram_fast_label['spectrogram_fast_label']  

spectrogram_slow_resized = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_resized.mat')
spectrogram_slow_resized = spectrogram_slow_resized['spectrogram_slow_resized']
spectrogram_slow_resized = np.transpose(spectrogram_slow_resized, (2, 0, 1))
spectrogram_slow_label = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_label.mat')
spectrogram_slow_label = spectrogram_slow_label['spectrogram_slow_label']  

spectrogram_slow_pocket_resized = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_pocket_resized.mat')
spectrogram_slow_pocket_resized = spectrogram_slow_pocket_resized['spectrogram_slow_pocket_resized']
spectrogram_slow_pocket_resized = np.transpose(spectrogram_slow_pocket_resized, (2, 0, 1))
spectrogram_slow_pocket_label = scipy.io.loadmat('/content/drive/MyDrive/spectrogram_slow_pocket_label.mat')
spectrogram_slow_pocket_label = spectrogram_slow_pocket_label['spectrogram_slow_pocket_label']  

In [ ]:
label_concat = np.concatenate((spectrogram_fast_label,spectrogram_slow_label),axis=0)
label_concat = np.concatenate((label_concat,spectrogram_slow_pocket_label),axis=0)

In [ ]:
# One-Hot Encode Label
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label_concat)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_labels = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
# Concat range-doppler data
range_doppler_concat = np.concatenate((range_doppler_fast_resized,range_doppler_slow_resized),axis=0)
range_doppler_concat = np.concatenate((range_doppler_concat,range_doppler_slow_pocket_resized),axis=0)
range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis] 
range_doppler_concat_label = label_concat
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
# Concat range-doppler data
spectrogram_concat = np.concatenate((spectrogram_fast_resized,spectrogram_slow_resized),axis=0)
spectrogram_concat = np.concatenate((spectrogram_concat,spectrogram_slow_pocket_resized),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis] 
spectrogram_concat_label = onehot_encoded_labels
# Shuffle concat range doppler
# shuffle_indx = random.sample(range(0, spectrogram_concat.shape[0]), spectrogram_concat.shape[0]) # split validation data
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

In [ ]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
def mixup_augmentation(images,range_doppler_training_data, labels, repeat_of_mixup, alpha=0.2):
    batch_size = images.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))
    
    for ii in range(repeat_of_mixup):
      # shuffle train dataset
      shuffle_indx_1 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_1 = images[shuffle_indx_1,:,:,:]
      range_doppler_training_data_shuffled_1 = range_doppler_training_data[shuffle_indx_1,:,:,:]
      labels_shuffled_1 = labels[shuffle_indx_1,:]

      shuffle_indx_2 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_2 = images[shuffle_indx_2,:,:,:]
      range_doppler_training_data_shuffled_2 = range_doppler_training_data[shuffle_indx_2,:,:,:]
      labels_shuffled_2 = labels[shuffle_indx_2,:]

      # Sample lambda and reshape it to do the mixup
      ll = np.random.beta(alpha, alpha, (batch_size,1,1,1))
      x_l = np.reshape(ll, (batch_size,1,1,1))
      y_l = np.reshape(ll, (batch_size,1))
      
      # Perform mixup on both images and labels by combining a pair of images/labels
      images_mixup = images_shuffled_1 * x_l + images_shuffled_2 * (1 - x_l)
      images_mixup_range_doppler = range_doppler_training_data_shuffled_1 * x_l + range_doppler_training_data_shuffled_2 * (1 - x_l)
      labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)
      concat_images[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup
      concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
      concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

    concat_images[repeat_of_mixup*batch_size:,:,:,:] = images
    concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = range_doppler_training_data
    concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images,concat_images_range_doppler, concat_label)

def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,range_doppler_concat_label_shuffle_train,\
                                       repeat_of_mixup, augmentation_enable):
  # ---------------- Parameters ----------------
  repeat_of_augmentation = 3
  size_of_validation = 30
  # Split validation
  randomlist_for_validation_indx = random.sample(range(0, range_doppler_concat_shuffle_train.shape[0]), size_of_validation) # split validation data
  randomlist_for_train_indx = np.delete(range(0, range_doppler_concat_shuffle_train.shape[0]), randomlist_for_validation_indx) # split training data
  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_labels = range_doppler_concat_label_shuffle_train[randomlist_for_validation_indx,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]

  (spectrogram_augmented_image,range_doppler_augmented_image,spectrograms_label)=\
   mixup_augmentation(spectrogram_training_data,range_doppler_training_data, spectrogram_training_labels, repeat_of_mixup, alpha=0.2)

  return (spectrogram_augmented_image,range_doppler_augmented_image,spectrograms_label,\
     spectrogram_validation_data,range_doppler_validation_data, spectrogram_validation_labels)



def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


In [ ]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)

In [ ]:
ii = 15
plt.figure()
plt.imshow(range_doppler_concat_shuffle[ii,:,:,0])
print(range_doppler_concat_label_shuffle[ii,:])
plt.figure()
plt.imshow(spectrogram_concat_shuffle[ii,:,:,0])
print(spectrogram_concat_label_shuffle[ii,:])

# plt.figure()
# plt.imshow(spectrogram_slow_pocket_resized[ii,:,:])
# print(spectrogram_slow_pocket_label[ii,:])
# plt.figure()
# plt.imshow(range_doppler_fast_resized[ii,:,:])
# print(range_doppler_fast_label[ii,:])
# plt.figure()
# plt.imshow(range_doppler_slow_resized[ii,:,:])
# print(range_doppler_slow_label[ii,:])
# plt.figure()
# plt.imshow(range_doppler_slow_pocket_resized[ii,:,:])
# print(range_doppler_pocket_label[ii,:])

In [ ]:
t = time.time()
# ---------- Parameters ----------------
augmentation_enable = True
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 100
batch_size = 32
dense_size = 32
dropout_prob_cnn = 0.1
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 5
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  for train, test in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):   
    gc.collect()
    K.clear_session()

    randomlist_for_test_indx = test
    randomlist_for_train_indx = train
    # test data
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    #train data
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    # ---------------- Split labels to equal them during augmentation for Validation ----------------
    (spectrogram_augmented_image,range_doppler_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_spectrogram,validation_range_doppler, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train,\
                                         repeat_of_mixup, augmentation_enable)

    def encoder_for_range_doppler(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      return Model(input, x)

    def encoder_for_spectrogram(input_shape):
      input = Input(shape=input_shape)
      x = Conv2D(4, (3, 3),padding='same',kernel_initializer="glorot_normal")(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(8, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal")(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_cnn)(x)
      x = MaxPooling2D(pool_size=(2, 2))(x)
      x = Flatten()(x)
      return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(128)(input)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(dense_size)(x)
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(3, activation="softmax")(x)
      return Model(input, x)
    input_shape = range_doppler_concat_shuffle.shape[1:]
    base_network_range_doppler = encoder_for_range_doppler(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_range_doppler(range_doppler_input)

    input_shape = spectrogram_concat_shuffle.shape[1:]
    base_network_spectrogram = encoder_for_spectrogram(input_shape) 
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_spectrogram(spectrogram_input)

    concat_layer = Concatenate()([processed_range_doppler, processed_spectrogram])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1]))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[range_doppler_input, spectrogram_input], outputs=[out])

   
    # print(model.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=35, verbose=0,restore_best_weights=True, mode='min')
    # earlyStopping = EarlyStopping(monitor='val_accuracy', patience=15, verbose=0,restore_best_weights=True, mode='max')
    history = model.fit((range_doppler_augmented_image,spectrogram_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler,validation_spectrogram) , (spectrogram_validation_labels)))
    tf.keras.models.load_model
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test],\
                                               [spectrogram_concat_label_shuffle_test],
                  batch_size=batch_size)
    # ---------------- Get Test Results ----------------
    y_test_predicted = model.predict((range_doppler_concat_shuffle_test,spectrogram_concat_shuffle_test), batch_size=batch_size)
    # ----- Binarize y_test_predicted values -----
    y_test_predicted_binary = np.zeros(y_test_predicted.shape[0])
    for ii in range(y_test_predicted.shape[0]):
      if argmax(y_test_predicted[ii,:]) == 0:
        y_test_predicted_binary[ii] = 0
      elif argmax(y_test_predicted[ii,:]) == 1:
        y_test_predicted_binary[ii] = 1
      else:
        y_test_predicted_binary[ii] = 2
    range_doppler_concat_label_shuffle_test1 = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test1, y_test_predicted_binary, average='macro')
    # ----------------------------------------------------------------------------------------------------------
    test_accuracy_per_fold.append(test_accuracy)
    f1_score_per_fold.append(test_f1_score)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  print(f1_score_per_run)

print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.2f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.2f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.2f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.2f}".format(np.std(f1_score_per_run, axis=0))}')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

In [ ]:
print(f'Mean test accuracy is {"{:.2f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.2f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.2f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.2f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.2f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.2f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.2f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.2f}".format(np.std(f1_score_per_run, axis=0))}')
print(f'Time elapsed through all process: {"{:.2f}".format(elapsed)}, sec')

Mean test accuracy is 0.65, mean test f1 score is 0.12, max test accuracy is 0.68, max test f1 score is 0.14, min test accuracy is 0.60, min test f1 score is 0.10, std of test accuracy is 0.03, std of test f1 score is 0.02
Time elapsed through all process: 3897.19, sec
